# Предсказание стоимости жилья

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.
В колонках датасета содержатся следующие данные:

•	longitude — широта;

•	latitude — долгота;

•	housing_median_age — медианный возраст жителей жилого массива;

•	total_rooms — общее количество комнат в домах жилого массива;

•	total_bedrooms — общее количество спален в домах жилого массива;

•	population — количество человек, которые проживают в жилом массиве;

•	households — количество домовладений в жилом массиве;

•	median_income — медианный доход жителей жилого массива;

•	median_house_value — медианная стоимость дома в жилом массиве;

•	ocean_proximity — близость к океану.

Постройте две модели линейной регрессии на разных наборах данных:

- используя все данные из файла;

- используя только числовые переменные, исключив категориальные.


Сравните результаты работы линейной регрессии на двух наборах данных по метрикам RMSE, MAE и R2. Сделайте выводы.

In [1]:
import pandas as pd 
import numpy as np

import pyspark
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler, OneHotEncoder
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline

RANDOM_SEED = 12345

## Подготовка данных

###	Инициализируем локальную Spark-сессию

In [2]:
spark = SparkSession.builder\
                    .master("local")\
                    .appName("EDA California Housing")\
                    .getOrCreate() 

### Прочитаем содержимое файла /datasets/housing.csv. с помощью pySpark. 

In [3]:
df = spark.read.load('/datasets/housing.csv', format='csv', inferSchema=True, header='true')
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



### Выведем названия колонок и их типы данных в виде таблицы, используя атрибут dtypes.

In [4]:
print(pd.DataFrame(df.dtypes, columns=['column', 'type']))

               column    type
0           longitude  double
1            latitude  double
2  housing_median_age  double
3         total_rooms  double
4      total_bedrooms  double
5          population  double
6          households  double
7       median_income  double
8  median_house_value  double
9     ocean_proximity  string


### Из полученной таблици видно, что во всех столбцах значения числовые, кроме ocean_proximity. Изучим уникальные знаения в данном столбце.

In [5]:
df.groupBy('ocean_proximity').count().orderBy('count', ascending=False).toPandas()

,ocean_proximity,count
0,<1H OCEAN,9136
1,INLAND,6551
2,NEAR OCEAN,2658
3,NEAR BAY,2290
4,ISLAND,5


Из полученных данных видно, что только пять жилых массивов располагаются на островах, больше всего жилых массивов расположено в категрии <1H OCEAN.

###  Выведем ввиде таблицы в Pandas базовые описательные статистики данных с помощью метода describe()

In [7]:
df.summary().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,25%,-121.8,33.93,18.0,1447.0,296.0,787.0,280.0,2.5625,119600.0,None
5,50%,-118.49,34.26,29.0,2127.0,435.0,1166.0,409.0,3.5347,179700.0,None
6,75%,-118.01,37.71,37.0,3146.0,647.0,1724.0,605.0,4.7426,264700.0,None
7,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


### Проверим наличие прорпусков в столбцах. 

In [8]:
columns = df.columns
for column in columns:
    print(column, df.where(F.isnan(column) | F.col(column).isNull()).count())


longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 207
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


В столбце total_bedrooms 207 пропусков, заполним их медианой. Для начало определим значение медианы.

In [9]:
total_bedrooms_median = df.approxQuantile('total_bedrooms', [0.5], 0)[0]

total_bedrooms_median

435.0

In [10]:
df = df.fillna(total_bedrooms_median, subset=['total_bedrooms'])

### Проверим наличие пропусков.

In [11]:
columns = df.columns
for column in columns:
    print(column, df.where(F.isnan(column) | F.col(column).isNull()).count())

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 0
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


### Разделим колонки на числовые и текстовые и определим целевой признак

In [12]:
categorical_cols = 'ocean_proximity'
numerical_cols = [col for col in df.columns if col != 'ocean_proximity' and col !='median_house_value']
target = 'median_house_value'
numerical_cols

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income']

### Создаем шаги для преобразования категориальных колонок, которые кодируем с помощью OneHotEncoder

In [16]:
stages = []
indexer = StringIndexer(inputCol=categorical_cols, outputCol=categorical_cols + '_idx')
encoder = OneHotEncoder(inputCol='ocean_proximity_idx', outputCol=categorical_cols +'_ohe')
categorical_assembler = VectorAssembler(inputCols=['ocean_proximity_ohe'], outputCol='categorical_features')
stages += [indexer, encoder, categorical_assembler]

### Создаем шаги для преобразования числовых признаков, которые стандартизируем с помощью StandardScaler

In [18]:
numerical_assembler = VectorAssembler(inputCols=numerical_cols, outputCol="numerical_features")
standardScaler = StandardScaler(inputCol='numerical_features', outputCol="numerical_features_scaled")
all_features = ['categorical_features', 'numerical_features_scaled']
final_assembler = VectorAssembler(inputCols=all_features, outputCol="features")
stages += [numerical_assembler, standardScaler, final_assembler ]

In [19]:
stages

[StringIndexer_70e540a84d23,
 OneHotEncoder_02656dbd66ce,
 VectorAssembler_e30cdcc6f05d,
 VectorAssembler_0e54c6ad74d9,
 StandardScaler_0e0ced3cc76d,
 VectorAssembler_3b3f20051881]

На данном этапе проведено следующее:
- Инициализировали локальную Spark-сессию и прочитали файл. Датасет состоит из 10 колонок и 20640 строк. 
- В колонке total_bedrooms обнаружены пропуски, данные пропуски заменили на медиану.
- Создали шаги для преобразования категориальных колонок, которые кодируем с помощью OneHotEncoder 
- Создаем шаги для преобразования числовых признаков, которые стандартизируем с помощью StandardScaler

## Обучение моделей

###  Прежде чем приступить к обучению модели, разделяем датасет на обучающую и тестовую выборки с помощью метода randomSplit()

In [24]:
train_data, test_data = df.randomSplit([.8,.2], seed=RANDOM_SEED)
print(train_data.count(), test_data.count()) 


16431 4209


### Сначала обучим модель линейной регресии, используя все данные из файла.

In [26]:
lr_all_features = LinearRegression(labelCol=target, featuresCol='features')
stages_1 = stages + [lr_all_features]
stages_1

[StringIndexer_70e540a84d23,
 OneHotEncoder_02656dbd66ce,
 VectorAssembler_e30cdcc6f05d,
 VectorAssembler_0e54c6ad74d9,
 StandardScaler_0e0ced3cc76d,
 VectorAssembler_3b3f20051881,
 LinearRegression_3d41a73e2dcd]

#### Создаем pipeline для обучения первой модели, обучаем модель и делаем предсказания на тестовой вывборке.

In [27]:
pipeline_all = Pipeline(stages=stages_1)

In [28]:
model_1 = pipeline_all.fit(train_data)

23/08/24 18:14:08 WARN Instrumentation: [c19b2df0] regParam is zero, which might cause numerical instability and overfitting.
23/08/24 18:14:08 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/08/24 18:14:08 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/08/24 18:14:09 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/08/24 18:14:09 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


#### Сохраним датасет с предсказаниями первой модели в переменной - predictions_all_features

In [30]:
predictions_all_features = model_1.transform(test_data)

In [31]:
predictions_col = 'prediction'

###  Теперь обучим модель линейной регресии, используя только числовые признаки.

In [33]:
lr_numeric_features = LinearRegression(labelCol=target, featuresCol='numerical_features_scaled')

In [34]:
stages_2 = stages + [lr_numeric_features]
stages_2

[StringIndexer_70e540a84d23,
 OneHotEncoder_02656dbd66ce,
 VectorAssembler_e30cdcc6f05d,
 VectorAssembler_0e54c6ad74d9,
 StandardScaler_0e0ced3cc76d,
 VectorAssembler_3b3f20051881,
 LinearRegression_70f941562e3f]

#### Создаем pipeline для обучения второй модели, обучаем модель и делаем предсказания на тестовой вывборке.

In [35]:
pipeline_col = Pipeline(stages=stages_2)
model_2 = pipeline_col.fit(train_data)

23/08/24 18:14:13 WARN Instrumentation: [56288805] regParam is zero, which might cause numerical instability and overfitting.


#### Сохраним датасет с предсказаниями второй модели в переменной - predictions_numeric_features

In [36]:
 predictions_numeric_features = model_2.transform(test_data)

На данном этапе мы разбили данные на две выборки, обучающую и тестовую в соотношении 80% - 20%. Провели обучениен двух моделей логической регрессии, одну их них обучали на всех данных, другую только на числовых признаках. Получили предсказания обеих моделей. Теперь необходимо оценить результьаты.

## Анализ результатов

Сравним результаты работы линейной регрессии на двух наборах данных по метрикам RMSE, MAE и R2. 

### Для анализа результатов работы модели создадим объект класса RegressionEvaluator:

In [37]:
evaluator = RegressionEvaluator(predictionCol=predictions_col, labelCol=target)

### Расчитаем метрику RMSE для обоих моделей.

In [38]:
print('RMSE model:', evaluator.evaluate(predictions_all_features, {evaluator.metricName: "rmse"}))
print('RMSE model_2:', evaluator.evaluate(predictions_numeric_features, {evaluator.metricName: "rmse"}))

RMSE model: 67710.876101571
RMSE model_2: 68813.70677046082


### Расчитаем метрику MAE для обоих моделей.

In [39]:
print('MAE model:', evaluator.evaluate(predictions_all_features, {evaluator.metricName: "mae"}))
print('MAE model_2:', evaluator.evaluate(predictions_numeric_features, {evaluator.metricName: "mae"}))

MAE model: 49104.5052138644
MAE model_2: 50311.01785165393


### Расчитаем метрику R2 для обоих моделей.

In [40]:
print('R2 model:', evaluator.evaluate(predictions_all_features, {evaluator.metricName: "r2"}))
print('R2 model_2:', evaluator.evaluate(predictions_numeric_features, {evaluator.metricName: "r2"}))

R2 model: 0.6588338487963326
R2 model_2: 0.6476299606813949


In [ ]:
spark.stop()

Вывод: Из полученных метрик для обеих моделей мы видим, что лучшие показетели у модели линейной регресии орбученной на всех данных (категориальных и числовых). RMSE и MAE первой модели меньше, чем у второй. А R2 выше у первой модели, чем у второй.

**Итоговый вывод:**
Перед нами стояла задача обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе этих данных нужно было предсказать медианную стоимость дома в жилом массиве. Для оценки качества модели необходимо было испольхзлвать метрики RMSE, MAE и R2. 
Для решения данно задачи нами было следано следующее:
- Инициализирована локальная Spark-сессию и прочитан файл. 
- Изучив данные, увидели, что во всех столбцах кроме total_bedrooms нет пропусков. Пропуски в столбце total_bedrooms заменили медианой.
- Провели трансформирование и кодирование категориальных признаков с помощью StringIndexer и OneHotEncoder соответсвенно.
- Числовые признаки масштабировали с помощью StandardScaler предварительно объеденив числовые признаки в один вектор. 
- Затем объеденили числовые и категориальные признаки в один вектор с помощью VectorAssembler
- Для обучения модели данны разбили на две выборки: обучающую и тестовую в соотвношении 80 -20.
- Провели обучениен двух моделей логической регрессии, одну из них обучали на всех данных, другую только на числовых признаках. Получили предсказания.

При проведении анализа результатов обеих моделей были расчитаны следующие метрики:
1. RMSE

RMSE model: 67710.87610131004

RMSE model_2: 68813.70677023288

2. MAE

MAE model: 49104.50521339809

MAE model_2: 50311.01785124337

3. R2

R2 model: 0.6588338487989622

R2 model_2: 0.6476299606837294

Из полученных данных видно, что первая модель линейной регресии, обученная на все данных (категориальных и числовых) показала более хорошиен результаты,  чем вторая, обученная только гна чиловых признаках.